# Perceptron
## Problem 1.1
Training error for normal perceptron: 0.04036697247706422 (after 2 passes).


Test error for normal perceptron: 0.0610079575596817. (after 2 passes)


Training error for voted perceptron: 0.04128440366972477 (after 2 passes).


Test error for voted perceptron: 0.0610079575596817. (after 2 passes)


Training error for average perceptron: 0.05412844036697248. (after 2 passes)


Test error for average perceptron: 0.08222811671087533. (after 2 passes)

-----------------------------------------------------------------------


Training error for normal perceptron: 0.02110091743119266 (after 3 passes).


Test error for normal perceptron: 0.04509283819628647. (after 3 passes)


Training error for voted perceptron: 0.030275229357798167 (after 3 passes).


Test error for voted perceptron: 0.04509283819628647. (after 3 passes)


Training error for average perceptron: 0.03486238532110092. (after 3 passes)


Test error for average perceptron: 0.0610079575596817. (after 3 passes)

-----------------------------------------------------------------------


Training error for normal perceptron: 0.01926605504587156 (after 4 passes).


Test error for normal perceptron: 0.04774535809018567. (after 4 passes)


Training error for voted perceptron: 0.025688073394495414 (after 4 passes).


Test error for voted perceptron: 0.04509283819628647. (after 4 passes)


Training error for average perceptron: 0.031192660550458717. (after 4 passes)


Test error for average perceptron: 0.050397877984084884. (after 4 passes)

-----------------------------------------------------------------------

## Problem 1.2

The three highest coordinates in Wavg are (Dictionary Index, Value) : (439, 117371.0), (467, 72270.0), (204, 45353.0)


Corresponding wors are "file", "program" and "line"


The three lowest coordinates in Wavg are (Dictionary Index, Value) : (79, -294511.0), (470, -164072.0), (394, -153026.0


Corresponding wors are "he", "team" and "game"

-----------------------------------------------------------------------

## Problem 1.3

![Confusion Matrix](./img/confusion_mat.png)

(a) Label 5 has the highest accuracy.
(b) Label 3 has the lowest accuracy.
(c) The perceptron classifier most often mistakenly classifies an example in class 6 belonging to class 5.



In [1]:
import numpy as np
import time
import random
from collections import defaultdict
from copy import deepcopy

In [24]:
def read_file(filename):
        f = open(filename,'r')
        ls = []
        ls_label = [list() for _ in range(6)]
        lines = f.readlines()
        for line in lines:
            content = [int(x) for x in line.strip().split()]
            ls.append(content)
            label = content[-1]
            ls_label[label-1].append(content)
        f.close()

        return np.asarray(ls), ls_label
    
    
def get_data_set_sequential(c1, c2, train_all):
    rs = []
    for data in train_all:
        if data[-1] == c1:
            data[-1] = 1
            rs.append(data)
        elif data[-1] == c2:
            data[-1] = -1
            rs.append(data)
    return np.asarray(rs)

def get_data_multi(c, train_all):
    rs = []
    
    for data in train_all:
        if data[-1] == c:
            data[-1] = 1
        else:
            data[-1] = -1
        rs.append(data)
    return np.asarray(rs)
    
def get_data_set(c1, c2, ls_label):
    c1_train = np.asarray(ls_label[c1-1])
    c2_train = np.asarray(ls_label[c2-1])
    c1_train[:,-1] = 1
    c2_train[:,-1] = -1
    train_data = np.asarray(np.vstack((c1_train, c2_train)))
    np.random.shuffle(train_data)
    return train_data
    

class Perceptron:
    def __init__(self, train_all):
        self.train = train_all
        self.w = np.zeros(self.train.shape[1]-1)
    
        
    def train_normal(self, t):
        for i in range(t):
            for data in self.train:
                y = data[-1]
                x = data[:-1]
                if y * np.dot(self.w, x) <= 0:
                    self.w = self.w + y * x
                    
    def train_voted(self, t):
        self.counts = defaultdict(tuple)
        w = np.zeros(self.train.shape[1]-1)
        m = 1
        cm = 1
        self.counts[m] = (w, cm)
        
        
        for i in range(t):

            w = self.w
            for data in self.train:
                y = data[-1]
                x = data[:-1]
                if y * np.dot(w, x) <= 0:
                    self.counts[m] = (w, cm)
                    w = w + y * x
                    m += 1
                    cm = 1
                else:
                    cm += 1
            self.w = w
            
    def train_average(self, t):
        self.counts = defaultdict(tuple)
        w = np.zeros(self.train.shape[1]-1)
        m = 1
        cm = 1
        self.counts[m] = (w, cm)
        
        self.w_sum = np.zeros(self.train.shape[1]-1)
        
        for i in range(t):

            w = self.w
            for data in self.train:
                y = data[-1]
                x = data[:-1]
                if y * np.dot(w, x) <= 0:
                    self.counts[m] = (w, cm)
                    self.w_sum += cm * w
                    w = w + y * x
                    m += 1
                    cm = 1
                else:
                    cm += 1
            self.w = w
        
    def evaluate_average(self, eval_data):
        eval_x = eval_data[:,:-1]
        eval_y = eval_data[:,-1]
        
        predictions = np.dot(eval_x, self.w_sum)
        assert eval_y.shape == predictions.shape, 'number of labels mismatch.'
        rs = np.where(predictions > 0, 1, -1)
        rs[np.where(predictions == 0)] = random.choice([0,1])
        return float(np.count_nonzero(eval_y - rs)) / eval_data.shape[0]
                                        
    def predict_voted(self, x):
        prediction = 0
        for m, (w, c) in self.counts.items():
            rs = np.dot(w, x)
            if rs > 0:
                prediction += c
            elif rs < 0:
                prediction -= c
        
        return 1 if prediction > 0 else -1
    
    def evaluate_voted(self, eval_data):
        eval_x = eval_data[:,:-1]
        eval_y = eval_data[:,-1]
        
        error = 0
        for data in eval_data:
            prediction = self.predict_voted(data[:-1])
            if prediction != data[-1]:
                error+=1
                
        return float(error) / eval_data.shape[0]
    
    def predict(self, x):
        return 1 if np.dot(x, self.w) > 0 else -1
        
    def predict_average(self, x):
        return 1 if np.dot(x, self.w_sum) > 0 else -1
    
    def evaluate(self, eval_data):
        eval_x = eval_data[:,:-1]
        eval_y = eval_data[:,-1]
        
        predictions = np.dot(eval_x, self.w)
        assert eval_y.shape == predictions.shape, 'number of labels mismatch.'
        rs = np.where(predictions > 0, 1, -1)
        rs[np.where(predictions == 0)] = random.choice([0,1])
        return float(np.count_nonzero(eval_y - rs)) / eval_data.shape[0]
    
    def reset(self):
        self.w = np.zeros(self.train.shape[1]-1)
        
    def get_w_sum(self):
        return self.w_sum
        
        

In [25]:
# Problem 1.1

train_all, train_separate = read_file('./data/pa3train.txt')
test_all, test_separate = read_file('./data/pa3test.txt')

times = 1
passes = 1

print('dimension of all train data: {}.'.format(train_all.shape))
#print('dimension of separate train data: {}.'.format(train_separate.shape))
train_data_seq = get_data_set_sequential(1,2,train_all)
train_data = get_data_set(1,2, train_separate)
test_data = get_data_set(1,2, test_separate)


# Normal Perceptron
p1 = Perceptron(train_data_seq)

p1.train_normal(passes)

train_error = p1.evaluate(train_data)
print('Training error for normal perceptron: {} (after {} passes).'.format(train_error, passes))

test_error = p1.evaluate(test_data)
print('Test error for normal perceptron: {}. (after {} passes)'.format(test_error, passes))

# Used only for shuffled data. Not useful for this grading

# train_error = 0.0
# for i in range(times):
#     train_error += p1.evaluate(train_data)
# train_error /= times
# print('The average training error is: {} (after running {} times).'.format(train_error, times))


# test_error = 0.0
# for i in range(times):
#     test_error += p1.evaluate(test_data)
# test_error /= times
# print('The average test error is: {} (after running {} times).'.format(test_error, times))

print()

# Voted Perceptron
p2 = Perceptron(train_data_seq)

p2.train_voted(passes)

train_error = p2.evaluate_voted(train_data)
print('Training error for voted perceptron: {} (after {} passes).'.format(train_error, passes))

test_error = p2.evaluate_voted(test_data)
print('Test error for voted perceptron: {}. (after {} passes)'.format(test_error, passes))

print()

# Average Perceptron

p3 = Perceptron(train_data_seq)

p3.train_average(passes)

train_error = p3.evaluate_average(train_data)
print('Training error for average perceptron: {}. (after {} passes)'.format(train_error, passes))

test_error = p3.evaluate_average(test_data)
print('Test error for average perceptron: {}. (after {} passes)'.format(test_error, passes))



dimension of all train data: (3000, 820).
Training error for normal perceptron: 0.04128440366972477 (after 1 passes).
Test error for normal perceptron: 0.05305039787798409. (after 1 passes)

Training error for voted perceptron: 0.06788990825688074 (after 1 passes).
Test error for voted perceptron: 0.08753315649867374. (after 1 passes)

Training error for average perceptron: 0.0798165137614679. (after 1 passes)
Test error for average perceptron: 0.11671087533156499. (after 1 passes)


In [26]:
# Problem 1.2

w_avg = p3.get_w_sum()

rs = []

for i,each in enumerate(list(w_avg)):
    rs.append((i+1, each))
sorted_rs = sorted(rs, key=lambda x: x[1], reverse=True)

for i in range(3):
    print(sorted_rs[i])
    
reverse_sorted_rs = sorted_rs[::-1]

for i in range(3):
    print(reverse_sorted_rs[i])





(439, 117371.0)
(467, 72270.0)
(204, 45353.0)
(79, -71608.0)
(470, -35978.0)
(142, -34471.0)


In [29]:
# Problem 1.3
confusion_mat = np.zeros((7,6))
train_all, _ = read_file('./data/pa3train.txt')
plist = []
for i in range(6):
    train_data = get_data_multi(i+1, deepcopy(train_all))
    p = Perceptron(train_data)
    p.train_normal(1)
    plist.append(p)

    
# for data in plist[1].train:
#     print(data[-1])
#print(plist[1].train)
    
# for data in train_data:    
#     print(plist[1].predict(data[:-1]))

    
    
N = defaultdict(int)
C = defaultdict(int)

for j in test_all[:,-1]:
    N[j] += 1
    
result = []
for data in test_all:
    test_x = data[:-1]
    test_y = data[-1]
    predictions = []
    for p in plist:
        predictions.append(p.predict(test_x))
    
    #print(predictions)
    if predictions.count(1) > 1:
        predictions = 7
    elif predictions.count(1) == 0:
        predictions = 7
    elif predictions.count(1) == 1:
        predictions = predictions.index(1) + 1
        
    #if predictions != test_y:
    C[(predictions, test_y)] += 1
    result.append(predictions)
    
result = np.asarray(result)
#print(result, test_all[:,-1])

for i in range(confusion_mat.shape[0]):
    for j in range(confusion_mat.shape[1]):
        confusion_mat[i][j] = float(C[(i+1,j+1)]) / float(N[j+1])
                            
print('Confusion matrix: ')                             
print(confusion_mat)
#print(len(plist))

Confusion matrix: 
[[ 0.71891892  0.00520833  0.03428571  0.02173913  0.          0.        ]
 [ 0.01081081  0.65625     0.03428571  0.02717391  0.01282051  0.01851852]
 [ 0.          0.015625    0.37142857  0.          0.          0.02777778]
 [ 0.01621622  0.00520833  0.          0.69021739  0.          0.        ]
 [ 0.01621622  0.03125     0.07428571  0.00543478  0.80128205  0.12037037]
 [ 0.00540541  0.01041667  0.03428571  0.          0.07051282  0.49074074]
 [ 0.23243243  0.27604167  0.45142857  0.25543478  0.11538462  0.34259259]]
